In [4]:
from lib.fablib_utils import slice_builder_utils
import os

In [5]:
def extend_slice_leases():
    slices = ["Kiran_Integrated_Test_1", "Kiran_P4_Test_1", "Final_Integrated_Topology_1", "Final_Integrated_Topology_2", "Final_Integrated_Topology_3"]
    for slice_name in slices:
        slice_builder_utils.extend_slice_lease(slice_name, days=14)

extend_slice_leases()

slice_builder_utils - New Lease End Date for Slice: Kiran_Integrated_Test_1 is 2022-12-12 15:35:20 +0000
slice_builder_utils - New Lease End Date for Slice: Kiran_P4_Test_1 is 2022-12-12 15:35:35 +0000
slice_builder_utils - New Lease End Date for Slice: Final_Integrated_Topology_1 is 2022-12-12 15:35:50 +0000
slice_builder_utils - New Lease End Date for Slice: Final_Integrated_Topology_2 is 2022-12-12 15:36:16 +0000
slice_builder_utils - New Lease End Date for Slice: Final_Integrated_Topology_3 is 2022-12-12 15:36:49 +0000


In [3]:
def get_node_interface_info(inst_slice, avoid_networks=[]):
    node_interface_info = {}
    interfaces = slice_builder_utils.get_slice_interfaces(slice=inst_slice)
    for interface in interfaces:
        node, net = interface.get_node(), interface.get_network().get_name()
        os_int, ip_addr, mac_addr = interface.get_os_interface(), interface.get_ip_addr(), interface.get_mac()
        if net in avoid_networks:
            pass
        elif node in node_interface_info:
            node_interface_info[node][net] = {"os_interface": os_int, "ip_addr": ip_addr, "mac_addr": mac_addr}
        else:
            node_interface_info[node] = {net: {"os_interface": os_int, "ip_addr": ip_addr, "mac_addr": mac_addr}}
    
    for i, src_node in enumerate(node_interface_info):
        for connection in node_interface_info[src_node]:
            for j, dst_node in enumerate(node_interface_info):
                if i == j:
                    pass
                elif connection in node_interface_info[dst_node]:
                    node_interface_info[src_node][connection]['dst_node'] = dst_node
                else:
                    pass
                
    return node_interface_info

In [4]:
slice_name = "Final_Integrated_Topology_3"
inst_slice = slice_builder_utils.get_slice_by_name_or_id(slice_name=slice_name)
avoid_networks = ["_meas_net"]
node_interface_info = get_node_interface_info(inst_slice, avoid_networks)

In [5]:
def get_ethernet_port_mappings(node_interface_info):
    ethernet_port_mappings = {}
    for node in node_interface_info:
        # print("{}:".format(node.get_name()))
        # print(node_interface_info[node])
        mappings = [node_interface_info[node][connection]['os_interface'] for connection in node_interface_info[node]]
        mappings.sort()
        final_mappings = {mappings[i]: i for i in range(len(mappings))}            
        ethernet_port_mappings[node.get_name()] = final_mappings
    print("Ethernet Port Mappings:", ethernet_port_mappings)
    return ethernet_port_mappings

In [6]:
def get_p4_run_command_for_switch(inst_slice, switch, ethernet_port_mappings, thrift_port=9030):
    command_prefix = "sudo simple_switch "
    thrift_port_command = "--thrift-port " + str(thrift_port) + " "
    interfaces_prefix = "--interface "
    interfaces_command = ""
    program_name = "p4_basic_routing_2"
    program_command = program_name + ".bmv2/" + program_name + ".json"
    for interface in ethernet_port_mappings[switch]:
        interfaces_command += interfaces_prefix + str(ethernet_port_mappings[switch][interface]) + "@" + str(interface) + " "
    final_command = command_prefix + thrift_port_command + interfaces_command + program_command
    print(switch, final_command)
    return final_command

def start_p4_on_switch(switch_commands):
    switch_nodes = [x for x in inst_slice.get_nodes() if x.get_name().startswith("Switch")]
    for switch in switch_nodes:
        print("Switching on P4 for Switch:", switch.get_name())
        switch.execute_thread(switch_commands[switch.get_name()])
        
def run_p4_on_topology(inst_slice, ethernet_port_mappings, thrift_port=9030):
    switch_nodes = [x.get_name() for x in inst_slice.get_nodes() if x.get_name().startswith("Switch")]
    print("Switch Nodes", switch_nodes)
    final_commands = {}
    for switch in switch_nodes:
        final_commands[switch] = get_p4_run_command_for_switch(inst_slice, switch, ethernet_port_mappings, thrift_port)
    start_p4_on_switch(final_commands)

In [7]:
for node in node_interface_info:
    print("{}:".format(node.get_name()))
    print(node_interface_info[node])

ethernet_port_mappings = get_ethernet_port_mappings(node_interface_info)
thrift_port = 9030
run_p4_on_topology(inst_slice, ethernet_port_mappings, thrift_port=thrift_port)

Host_1:
{'HS1': {'os_interface': 'ens8', 'ip_addr': '192.168.1.2', 'mac_addr': '42:3B:6C:9A:8B:D3', 'dst_node': <fabrictestbed_extensions.fablib.node.Node object at 0x7fb62c38d0a0>}}
Host_2:
{'HS2': {'os_interface': 'ens7', 'ip_addr': '192.168.2.2', 'mac_addr': '42:C9:CB:D1:F4:E4', 'dst_node': <fabrictestbed_extensions.fablib.node.Node object at 0x7fb62c38dc10>}}
Host_3:
{'HS3': {'os_interface': 'ens7', 'ip_addr': '192.168.3.2', 'mac_addr': '52:19:F9:23:13:54', 'dst_node': <fabrictestbed_extensions.fablib.node.Node object at 0x7fb62c38dd00>}}
Host_4:
{'HS4': {'os_interface': 'ens7', 'ip_addr': '192.168.4.2', 'mac_addr': '52:43:6D:88:36:00', 'dst_node': <fabrictestbed_extensions.fablib.node.Node object at 0x7fb62c38dd30>}}
Switch_1:
{'SS13': {'os_interface': 'ens7', 'ip_addr': '192.170.2.1', 'mac_addr': '52:71:45:BA:B8:81', 'dst_node': <fabrictestbed_extensions.fablib.node.Node object at 0x7fb62c38dd00>}, 'HS1': {'os_interface': 'ens8', 'ip_addr': '192.168.1.1', 'mac_addr': '56:AD:CD:A8

In [25]:
import random

def get_host_info(info, host):
    host_info = None
    for h in info:
        if h.get_name() == host:
            host_info = node_interface_info[h]
            break
    return host_info

def get_connection_info(info, endpoint):
    for connection in info:
        if info[connection]["dst_node"].get_name() == endpoint:
            return connection
        else:
            pass
    return None
    
def get_configuration_info(node_interface_info, src_host, dst_host):
    src_host_info = get_host_info(node_interface_info, src_host)
    dst_host_info = get_host_info(node_interface_info, dst_host)
    if src_host_info is None or dst_host_info is None:
        print("Cannot find Source Host {} / Destination Host {} in Node Interface Info".format(src_host, dst_host))
        return None
    else:
        pass
    
    src_dst_conn = get_connection_info(src_host_info, endpoint=dst_host)
    dst_src_conn = get_connection_info(dst_host_info, endpoint=src_host)
    if src_dst_conn is None or dst_src_conn is None or src_dst_conn != dst_src_conn:
        print("Cannot find Connection between Source {} / Destiantion Host {} in Node Interface Info".format(src_host, dst_host))
        return None
    else:
        final_connection = src_dst_conn
    
    src_egress_port = src_host_info[final_connection]['os_interface']
    dst_port_mac_addr = dst_host_info[final_connection]['mac_addr']
    dst_host_ip_addr = dst_host_info[final_connection]['ip_addr']
    return final_connection, dst_port_mac_addr, src_egress_port, dst_host_ip_addr

def get_final_route(all_routes, src_host, dst_host, pick_route=None):
    if src_host not in all_routes:
        print("No Routes available for Source Host:", src_host, "in Routes:", all_routes)
        return None
    if dst_host not in all_routes[src_host]:
        print("No Routes available to Destination Host:", dst_host, "from Source Host:", src_host, "in Routes:", all_routes)
        return None
    
    available_routes = all_routes[src_host][dst_host]
    n = len(available_routes)
    if n == 0: 
        print("Zero Routes available to Destination Host:", dst_host, "from Source Host:", src_host, "in Routes:", all_routes)
        return None
    if n == 1:
        final_route = available_routes[0]
    elif pick_route is None:  # Pick an Arbitrary Route if Multiple Routes found & no Preferences given
        route_no = random.randint(0, n - 1)
        final_route = available_routes[route_no]
    elif 0 < pick_route <= n:
        final_route = available_routes[pick_route - 1]
    else:
        print("Invalid Route Picked!")
        return None
        
    print("Using Route:", final_route)
    return final_route

def get_complete_p4_route_configurations(node_configuration_info, route):
    route_configurations = []
    for i in range(0, len(route) - 1, 1):
        src, dst = route[i], route[i + 1]
        if src.startswith("Host"):
            pass
        else:
            configuration = get_configuration_info(node_interface_info, src, dst)
            if configuration is None:
                print("Aborting Configutation")
                return None
            else:
                route_configurations.append([src] + list(configuration))
    print("Route Configurations for Route:", route, "->", route_configurations)
    return route_configurations

In [39]:
def get_ip_from_hex(hex_ip):
    if len(hex_ip) == 8:
        addr = []
        for i in range(0, len(hex_ip), 2):
            addr.append(str(int(hex_ip[i:i+2], base=16)))
        return ".".join(addr)
    else:
        print("Invalid IPv4 Address!!")
        return None
    
def parse_dump_entries(lines):
    ip_entries = {}
    curr_entry_handle = ""
    parse_next = False
    for line in lines:
        if parse_next is True:
            ip_addr_complete = line.split()[-1]
            ip_addr, ip_addr_subnet = ip_addr_complete.split("/")
            int_ip = ip_addr if "." in ip_addr else get_ip_from_hex(ip_addr)
            final_ip = int_ip + "/" + ip_addr_subnet
            ip_entries[final_ip] = curr_entry_handle 
            parse_next = False
        elif line.startswith("Dumping entry"):
            curr_entry_handle = line.split()[-1]
            curr_entry_handle = int(curr_entry_handle, base=16) if "x" in curr_entry_handle else int(curr_entry_handle)
        elif line.startswith("Match key:"):
            parse_next = True
        else:
            pass
    print("IP Entries:", ip_entries)
    return ip_entries

In [62]:
def execute_p4_table_dump_command(switch, table_name, thrift_port=9030):
    p4_command_prefix = "echo \"table_dump"
    p4_command_args = [p4_command_prefix, table_name, "\""]
    p4_run_command = "simple_switch_CLI --thrift-port " + str(thrift_port)
    p4_dump_command = " ".join(p4_command_args) + " | " + p4_run_command
    print("Executing P4 Table Dump Command:", p4_dump_command, "on Switch:", switch.get_name())
    stdout = switch.execute(p4_dump_command)
    return stdout[0].split("\n")

def execute_p4_add_entry_command(switch, table_name, action_name, addr, params, thrift_port=9030):
    p4_command_prefix = "echo \"table_add"
    p4_command_args = [p4_command_prefix, table_name, action_name, addr, "=>"] + params + ["\""]
    p4_run_command = "simple_switch_CLI --thrift-port " + str(thrift_port)
    p4_add_command = " ".join(p4_command_args) + " | " + p4_run_command
    print("Executing P4 Add Command:", p4_add_command, "on Switch:", switch.get_name())
    stdout = switch.execute(p4_add_command)
    print("Add Command Result:", stdout)

def execute_p4_delete_entry_command(switch, table_name, handle_id, thrift_port=9030):
    p4_command_prefix = "echo \"table_modify"
    p4_command_args = [p4_command_prefix, table_name, handle_id] + ["\""]
    p4_run_command = "simple_switch_CLI --thrift-port " + str(thrift_port)
    p4_delete_command = " ".join(p4_command_args) + " | " + p4_run_command
    print("Executing P4 Delete Command:", p4_delete_command, "on Switch:", switch.get_name())
    stdout = switch.execute(p4_delete_command)
    print("Delete Command Result:", stdout)
        
def execute_p4_modify_entry_command(switch, table_name, action_name, handle_id, params, thrift_port=9030):
    p4_command_prefix = "echo \"table_modify"
    p4_command_args = [p4_command_prefix, table_name, action_name, handle_id] + params + ["\""]
    p4_run_command = "simple_switch_CLI --thrift-port " + str(thrift_port)
    p4_modify_command = " ".join(p4_command_args) + " | " + p4_run_command
    print("Executing P4 Modify Command:", p4_modify_command, "on Switch:", switch.get_name())
    stdout = switch.execute(p4_modify_command)
    print("Modify Command Result:", stdout)

In [63]:
def execute_p4_route_configurations(inst_slice, p4_route_configurations, dst_ip_addr_subnet, ethernet_port_mappings, thrift_port=9030):
    switches = inst_slice.get_nodes()
    for p4_route_config in p4_route_configurations:
        switch_name, _, dst_mac_addr, egress_port, _ = p4_route_config
        switch = [s for s in switches if s.get_name() == switch_name][0]
        params = [dst_mac_addr, str(ethernet_port_mappings[switch_name][egress_port])]
        table_name, action_name = "MyIngress.ipv4_lpm", "MyIngress.ipv4_forward"
        table_dump_entries = execute_p4_table_dump_command(switch, table_name, thrift_port=thrift_port)
        table_ip_entries = parse_dump_entries(table_dump_entries)
        if dst_ip_addr_subnet in table_ip_entries:
            handle_id = str(table_ip_entries[dst_ip_addr_subnet])
            execute_p4_modify_entry_command(switch, table_name, action_name, handle_id, params, thrift_port)
        else:
            execute_p4_add_entry_command(switch, table_name, action_name, dst_ip_addr_subnet, params, thrift_port)
        
def add_complete_route(inst_slice, all_routes, node_interface_info, ethernet_port_mappings, src_host, dst_host, thrift_port, pick_route=None):
    route = get_final_route(all_routes, src_host, dst_host, pick_route)
    p4_route_configurations = get_complete_p4_route_configurations(node_interface_info, route)
    dst_ip_addr = p4_route_configurations[-1][-1]
    dst_ip_addr_subnet = ".".join(dst_ip_addr.split(".")[0:3]) + ".0/24"
    print("Destination IP Address: {} Destination IP Address Subnet {}".format(dst_ip_addr, dst_ip_addr_subnet))
    execute_p4_route_configurations(inst_slice, p4_route_configurations, dst_ip_addr_subnet, ethernet_port_mappings, thrift_port)

In [64]:
host_routes = {}
host_routes["Host_1"] = {
    "Host_2": [["Host_1", "Switch_1", "Switch_2", "Host_2"], ["Host_1", "Switch_1", "Switch_3", "Switch_4", "Switch_2", "Host_2"]],
    "Host_3": [["Host_1", "Switch_1", "Switch_3", "Host_3"], ["Host_1", "Switch_1", "Switch_2", "Switch_4", "Switch_3", "Host_3"]],
    "Host_4": [["Host_1", "Switch_1", "Switch_2", "Switch_4", "Host_4"], ["Host_1", "Switch_1", "Switch_3", "Switch_4", "Host_4"]]
}
host_routes["Host_2"] = {
    "Host_1": [["Host_2", "Switch_2", "Switch_1", "Host_1"], ["Host_2", "Switch_2", "Switch_4", "Switch_3", "Switch_1", "Host_1"]],
    "Host_4": [["Host_2", "Switch_2", "Switch_4", "Host_4"]]
}
host_routes["Host_3"] = {
    "Host_1": [["Host_3", "Switch_3", "Switch_1", "Host_1"], ["Host_3", "Switch_3", "Switch_4", "Switch_2", "Switch_1", "Host_1"]],
    "Host_4": [["Host_3", "Switch_3", "Switch_4", "Host_4"]]
}
host_routes["Host_4"] = {
    "Host_1": [["Host_4", "Switch_4", "Switch_2", "Switch_1", "Host_1"], ["Host_4", "Switch_4", "Switch_3", "Switch_1", "Host_1"]],
    "Host_2": [["Host_4", "Switch_4", "Switch_2", "Host_2"], ["Host_4", "Switch_4", "Switch_3", "Switch_1", "Switch_2", "Host_2"]],
    "Host_3": [["Host_4", "Switch_4", "Switch_3", "Host_3"], ["Host_4", "Switch_4", "Switch_2", "Switch_1", "Switch_3", "Host_3"]]
}

In [57]:
src_host, dst_host = "Host_3", "Host_4"
add_complete_route(inst_slice, host_routes, node_interface_info, ethernet_port_mappings, src_host, dst_host, thrift_port, pick_route=1)

src_host, dst_host = "Host_4", "Host_3"
add_complete_route(inst_slice, host_routes, node_interface_info, ethernet_port_mappings, src_host, dst_host, thrift_port, pick_route=1)

Using Route: ['Host_3', 'Switch_3', 'Switch_4', 'Host_4']
Route Configurations for Route: ['Host_3', 'Switch_3', 'Switch_4', 'Host_4'] -> [['Switch_3', 'SS34', '76:AB:BC:AA:83:67', 'ens10', '192.170.4.2'], ['Switch_4', 'HS4', '52:43:6D:88:36:00', 'ens9', '192.168.4.2']]
Destination IP Address: 192.168.4.2 Destination IP Address Subnet 192.168.4.0/24
Executing P4 Table Dump Command: echo "table_dump MyIngress.ipv4_lpm " | simple_switch_CLI --thrift-port 9030 on Switch: Switch_3
IP Entries: {'192.168.4.0/24': 0, '192.168.3.0/24': 1, '192.168.1.0/24': 2}
Executing P4 Modify Command: echo "table_modify MyIngress.ipv4_lpm MyIngress.ipv4_forward 0 76:AB:BC:AA:83:67 0 " | simple_switch_CLI --thrift-port 9030 on Switch: Switch_3
Modify Command Result: ('Obtaining JSON from switch...\nDone\nControl utility for runtime P4 table manipulation\nRuntimeCmd: Modifying entry 0 for lpm match table MyIngress.ipv4_lpm\nRuntimeCmd: \n', '')
Executing P4 Table Dump Command: echo "table_dump MyIngress.ipv4_

In [60]:
src_host, dst_host = "Host_1", "Host_4"
add_complete_route(inst_slice, host_routes, node_interface_info, ethernet_port_mappings, src_host, dst_host, thrift_port, pick_route=2)

src_host, dst_host = "Host_4", "Host_1"
add_complete_route(inst_slice, host_routes, node_interface_info, ethernet_port_mappings, src_host, dst_host, thrift_port, pick_route=2)

Using Route: ['Host_1', 'Switch_1', 'Switch_3', 'Switch_4', 'Host_4']
Route Configurations for Route: ['Host_1', 'Switch_1', 'Switch_3', 'Switch_4', 'Host_4'] -> [['Switch_1', 'SS13', '66:AC:7A:43:90:00', 'ens7', '192.170.2.2'], ['Switch_3', 'SS34', '76:AB:BC:AA:83:67', 'ens10', '192.170.4.2'], ['Switch_4', 'HS4', '52:43:6D:88:36:00', 'ens9', '192.168.4.2']]
Destination IP Address: 192.168.4.2 Destination IP Address Subnet 192.168.4.0/24
Executing P4 Table Dump Command: echo "table_dump MyIngress.ipv4_lpm " | simple_switch_CLI --thrift-port 9030 on Switch: Switch_1
IP Entries: {'192.168.4.0/24': 0, '192.168.1.0/24': 1}
Executing P4 Modify Command: echo "table_modify MyIngress.ipv4_lpm MyIngress.ipv4_forward 0 66:AC:7A:43:90:00 0 " | simple_switch_CLI --thrift-port 9030 on Switch: Switch_1
Modify Command Result: ('Obtaining JSON from switch...\nDone\nControl utility for runtime P4 table manipulation\nRuntimeCmd: Modifying entry 0 for lpm match table MyIngress.ipv4_lpm\nRuntimeCmd: \n', 

In [65]:
src_host, dst_host = "Host_1", "Host_4"
add_complete_route(inst_slice, host_routes, node_interface_info, ethernet_port_mappings, src_host, dst_host, thrift_port, pick_route=1)

src_host, dst_host = "Host_4", "Host_1"
add_complete_route(inst_slice, host_routes, node_interface_info, ethernet_port_mappings, src_host, dst_host, thrift_port, pick_route=1)

Using Route: ['Host_1', 'Switch_1', 'Switch_2', 'Switch_4', 'Host_4']
Route Configurations for Route: ['Host_1', 'Switch_1', 'Switch_2', 'Switch_4', 'Host_4'] -> [['Switch_1', 'SS12', '66:34:E5:92:21:E2', 'ens9', '192.170.1.2'], ['Switch_2', 'SS24', '6E:D6:0F:82:A6:C7', 'ens7', '192.170.3.2'], ['Switch_4', 'HS4', '52:43:6D:88:36:00', 'ens9', '192.168.4.2']]
Destination IP Address: 192.168.4.2 Destination IP Address Subnet 192.168.4.0/24
Executing P4 Table Dump Command: echo "table_dump MyIngress.ipv4_lpm " | simple_switch_CLI --thrift-port 9030 on Switch: Switch_1
IP Entries: {'192.168.4.0/24': 0, '192.168.1.0/24': 1}
Executing P4 Modify Command: echo "table_modify MyIngress.ipv4_lpm MyIngress.ipv4_forward 0 66:34:E5:92:21:E2 2 " | simple_switch_CLI --thrift-port 9030 on Switch: Switch_1
Modify Command Result: ('Obtaining JSON from switch...\nDone\nControl utility for runtime P4 table manipulation\nRuntimeCmd: Modifying entry 0 for lpm match table MyIngress.ipv4_lpm\nRuntimeCmd: \n', '